In [20]:
import pandas as pd
import numpy as np
import ast
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 

#해당 폴더 들어가기 전까지의 본인의 path 설정
PATH = 'c:\\Users\\passi\\Desktop\\programming\\stair_to_heaven\\'

#분봉 데이터 파일들을 모두 리스트에 넣기
file_list = os.listdir(PATH + "catch_highest/data/minute_stock_data/")
file_list.sort()
if '.DS_Store' in file_list:
    file_list = file_list[1:]
else:
    None

In [21]:
for i in range(len(file_list)):
    if file_list[i].startswith('20200401'):
        file_list = file_list[i:]
        break

In [22]:
for i in reversed(range(len(file_list))):
    if file_list[i].startswith('20200630'):
        file_list = file_list[:i+1]
        break

In [23]:
file_list

['20200401_KG ETS.csv',
 '20200401_갤럭시아컴즈.csv',
 '20200401_경농.csv',
 '20200401_바디텍메드.csv',
 '20200401_바이오니아.csv',
 '20200401_부광약품.csv',
 '20200401_부산산업.csv',
 '20200401_셀트리온헬스케어.csv',
 '20200401_손오공.csv',
 '20200401_시노펙스.csv',
 '20200401_아난티.csv',
 '20200401_아시아종묘.csv',
 '20200401_알서포트.csv',
 '20200401_앱클론.csv',
 '20200401_에스맥.csv',
 '20200401_에코마이스터.csv',
 '20200401_조비.csv',
 '20200401_지티지웰니스.csv',
 '20200401_진원생명과학.csv',
 '20200401_텔콘RF제약.csv',
 '20200401_푸른기술.csv',
 '20200401_한일진공.csv',
 '20200401_현대사료.csv',
 '20200401_효성오앤비.csv',
 '20200402_CJ씨푸드.csv',
 '20200402_넥스트BT.csv',
 '20200402_멕아이씨에스.csv',
 '20200402_바른손.csv',
 '20200402_바른손이앤에이.csv',
 '20200402_빙그레.csv',
 '20200402_알리코제약.csv',
 '20200402_애니플러스.csv',
 '20200402_우리들제약.csv',
 '20200402_우리들휴브레인.csv',
 '20200402_토박스코리아.csv',
 '20200402_플레이디.csv',
 '20200402_플레이위드.csv',
 '20200402_필로시스헬스케어.csv',
 '20200402_하이트론.csv',
 '20200402_한국테크놀로지.csv',
 '20200402_해태제과식품.csv',
 '20200403_나노엔텍.csv',
 '20200403_나무가.csv',
 '20200403_레몬.csv',
